In [1]:
from HeaderP import *
import pynbody as pb
import pynbody.filt as filt
import os, pickle, glob


In [5]:
def readsimdata(simNo,Rcut,Rcut_thread,zcut_thread,disk=True):
    if disk:
        data=pd.read_pickle('./g'+simNo+'_z_0_decomp.pkl')
    else:
        data=pd.read_pickle('./g'+simNo+'_z_0.pkl')
    data['R']=np.sqrt((data['x'])**2+data['y']**2)
    
    if Rcut:
        data=data.loc[data['Rbir']<Rcut_thread].reset_index(drop=True)
        data=data.loc[abs(data['zbir'])<zcut_thread].reset_index(drop=True)
    data['dR']=data['R']-data['Rbir']
    data['dz']=abs(data['z'])-abs(data['zbir'])
    if disk:
        data = data.loc[(data['decomp']==1)|(data['decomp']==4)].reset_index()
    return data


import matplotlib
from scipy import interpolate
from scipy.interpolate import interp1d
def getcolor(vrange,cm=plt.cm.get_cmap('viridis')):
    cmap = cm
    norm = matplotlib.colors.Normalize(vmin=min(vrange[:-1]), vmax=max(vrange[:-1]))
    return [cmap(norm(i)) for i in vrange]

In [6]:
import os 
sims = os.listdir('./')
sims = [i for i in sims if 'x' in i]
sims = [i for i in sims if '.txt' not in i]
sims = [i for i in sims if '.pkl' not in i]
simsnames = [i.split('_')[0][1:] for i in sims]


In [ ]:
from os.path import exists

cols = ['iord','r','z']
cmap = plt.cm.get_cmap('binary',20)

for i in range(len(sims)):
    if sims[i]=='1.12e12':
        continue
    print(sims[i])
    simname = sims[i]
    
    allsnaps = glob.glob(sims[i]+'/*.?????')
    
    agepickle = readsimdata(simsnames[i],True, 15, 5)
    
    allsnaps_time = [int(i.split('.')[-1]) for i in allsnaps if (('param' not in i) and ('draco' not in i) and ('dat_' not in i) and ('79e12' not in i.split('.')[-1]))]
    snaplookup = {int(allsnaps[i].split('.')[-1]):allsnaps[i] for i in range(len(allsnaps)) if (('param' not in allsnaps[i]) and ('draco' not in allsnaps[i]) and ('dat_' not in allsnaps[i]) and ('79e12' not in allsnaps[i].split('.')[-1]))}

    sortage = sorted(allsnaps_time)
    redshifts = np.zeros(len(sortage))
    allages_rs = np.zeros(len(redshifts))
    for j in trange(len(sortage)):
        s = pb.load(snaplookup[sortage[j]])
        h = s.properties['h']
        ages = float(str(s.properties['time']).split(' ')[0])
        allages_rs[j] = ages
            
    for j in trange(len(allages_rs)):
        print('reading ',snaplookup[sortage[j]])
        if exists(snaplookup[sortage[j]]+'.pkl'):
            this_snap = pd.read_pickle(snaplookup[sortage[j]]+'.pkl')
        else:
            s = pb.load(snaplookup[sortage[j]])
            h=s.halos()
            try:
                pb.analysis.angmom.faceon(h[1])
            except:
                print('no halo!')
                continue
            s.physical_units()
            sim = h[1]

            stars = sim.s
            this_snap = pd.DataFrame({k:stars[k] for k in cols})
            this_snap = pd.merge(this_snap, agepickle[['iord', 'age', 'feh', 'Rbir','zbir']], left_on='iord', right_on='iord', how='inner')
            this_snap.to_pickle(snaplookup[sortage[j]]+'.pkl')
        
        plt.figure(figsize=(10,8))
        plt.title(simsnames[i]+'; Cosmic Time = %.3f'%(allages_rs[j])+' Gyr', fontsize=25)
        plt.hexbin(this_snap['age'], this_snap['r'], C = this_snap['r']-this_snap['Rbir'],
                   extent = (0,14,0,15), gridsize=100, vmin=-5, vmax=5, cmap=cmap, reduce_C_function = np.std)
        plt.xlabel('Age [Gyr]')
        plt.ylabel('Current Radius')
        cb = plt.colorbar()
        cb.set_label('$σ_{R-R_{bir}}$ [kpc]')
        plt.tight_layout()
        plt.savefig('RM/'+simsnames[i]+'/'+str(allages_rs[j])+'.png')
        plt.close()

g6.96e11_3x9


  1%|          | 1/128 [00:00<00:17,  7.07it/s]

reading  g6.96e11_3x9/6.96e11.00008
no halo!
reading  g6.96e11_3x9/6.96e11.00016


  2%|▏         | 2/128 [00:00<00:36,  3.44it/s]

reading  g6.96e11_3x9/6.96e11.00024


...

reading  g6.96e11_3x9/6.96e11.00408


 40%|███▉      | 51/128 [1:12:46<3:39:16, 170.86s/it]

reading  g6.96e11_3x9/6.96e11.00416


 41%|████      | 52/128 [1:15:57<3:43:48, 176.69s/it]

reading  g6.96e11_3x9/6.96e11.00424
